In [3]:
import copy  # not used
import os
import sys
import time

import pickle

import numpy as np
import pandas as pd

from src.toric_model import Toric_code
from src.planar_model import Planar_code
from src.mcmc import *
from decoders import *
from src.mwpm import *


# This function generates training data with help of the MCMC algorithm
def generate(file_path, params, max_capacity=10**4, nbr_datapoints=10**6, fixed_errors=None):

    if params['code'] == 'planar':
        nbr_eq_class = 4
    elif params['code'] == 'toric':
        nbr_eq_class = 16

    if params['method'] == "all":
        nbr_eq_class *= 3
    # Creates data file if there is none otherwise adds to it
    try:
        df = pd.read_pickle(file_path)
        nbr_existing_data = df.index[-1][0] + 1
    except:
        df = pd.DataFrame()
        nbr_existing_data = 0

    print('\nDataFrame with ' + str(nbr_existing_data) +
          ' datapoints opened at: ' + str(file_path))

    # Stop the file from exceeding the max limit of nbr of datapoints
    nbr_to_generate = min(max_capacity-nbr_existing_data, nbr_datapoints)
    if nbr_to_generate < nbr_datapoints:
        print('Generating ' + str(max(nbr_to_generate, 0))
              + ' datapoins instead of ' + str(nbr_datapoints)
              + ', as the given number would overflow existing file')

    if fixed_errors != None:
        nbr_to_generate = 10000000
    failed_syndroms = 0

    df_list = []  # Initiate temporary list

    # Loop to generate data points
    for i in range(nbr_existing_data, nbr_existing_data + nbr_to_generate):
        print('Starting generation of point nr: ' + str(i + 1))

        # Initiate code
        if params['code'] == 'toric':
            init_code = Toric_code(params['size'])
            init_code.generate_random_error(params['p_error'])
        elif params['code'] == 'planar':
            init_code = Planar_code(params['size'])
            #print(type(init_code))
            init_code.generate_random_error(params['p_error'])

        # Flatten initial qubit matrix to store in dataframe
        df_qubit = copy.deepcopy(init_code.qubit_matrix)
        eq_true = init_code.define_equivalence_class()
    
        vertex_defects = init_code.vertex_defects
        plaquette_defects = init_code.plaquette_defects
        
        
        vertex_defects = 1*vertex_defects
        plaquette_defects = 1*plaquette_defects
        #print(vertex_defects)
        #print(plaquette_defects)
        
        
        pickle_defects = open("defect_matrices/s9p1/dict.defects","ab")
        pickle.dump((vertex_defects,plaquette_defects),pickle_defects)
        pickle_defects.close()
        

        
        
        #vdefect_matrix_file.close()
        #pdefect_matrix_file.close()
        
    ###    
        if params['mwpm_init']: #get mwpm starting points
            #print()
            init_code = class_sorted_mwpm(init_code)
            print('Starting in MWPM state')
        else: #randomize input matrix, no trace of seed.
            init_code.qubit_matrix, _ = init_code.apply_random_logical()
            init_code.qubit_matrix = init_code.apply_stabilizers_uniform()
            print('Starting in random state')

        # Generate data for DataFrame storage  OBS now using full bincount, change this
        if params['method'] == "PTEQ":
            df_eq_distr = PTEQ(init_code, params['p_error'])
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        if params['method'] == "PTDC":
            df_eq_distr, conv = PTDC(init_code, params['p_error'], params['p_sampling'])
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        if params['method'] == "PTRC":
            df_eq_distr, conv = PTRC(init_code, params['p_error'], params['p_sampling'])
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        elif params['method'] == "STDC":
            df_eq_distr = STDC(init_code, params['p_error'], params['p_sampling'], steps=params['steps'], droplets=params['droplets'])
            df_eq_distr = np.array(df_eq_distr)
            #print(df_eq_distr)
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        elif params['method'] == "ST":
            df_eq_distr = single_temp(init_code, params['p_error'], params['steps'])
            df_eq_distr = np.array(df_eq_distr)
            if np.argmin(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        elif params['method'] == "STRC":
            df_eq_distr = STRC(init_code, params['p_error'], p_sampling=params['p_sampling'], steps=params['steps'], droplets=params['droplets'])
            df_eq_distr = np.array(df_eq_distr)
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        elif params['method'] == "all":
            #init_code.qubit_matrix = init_code.apply_stabilizers_uniform()
            #df_eq_distr1 = single_temp(init_code, params['p_error'],params['steps'])

            #init_code.qubit_matrix = init_code.apply_stabilizers_uniform()
            
            choice = regular_mwpm(copy.deepcopy(init_code))
            df_eq_distr1 = np.zeros((4)).astype(np.uint8)
            df_eq_distr1[choice] = 100
            
            #df_eq_distr2 = PTEQ(init_code, params['p_error'])

            #init_code.qubit_matrix = init_code.apply_stabilizers_uniform()
            df_eq_distr3 = STDC(init_code, params['p_error'], p_sampling=params['p_sampling'], steps=params['steps'], droplets=params['droplets'])

            df_eq_distr = np.concatenate((df_eq_distr1,df_eq_distr3), axis=0)
        elif params['method'] == "eMWPM":
            out = class_sorted_mwpm(copy.deepcopy(init_code))
            lens = np.zeros((4))
            for j in range(4):
                lens[j] = out[j].count_errors()
            choice = np.argmin(lens)
            df_eq_distr = np.zeros((4)).astype(np.uint8)
            df_eq_distr[choice] = 100
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1
        elif params['method'] == "MWPM":
            #print(type(init_code))
            choice = regular_mwpm(copy.deepcopy(init_code))
            #choice = regular_mwpm(init_code_pre_mwpm)
            
            df_eq_distr = np.zeros((4)).astype(np.uint8)
            df_eq_distr[choice] = 100
            if np.argmax(df_eq_distr) != eq_true:
                print('Failed syndrom, total now:', failed_syndroms)
                failed_syndroms += 1

        # Generate data for DataFrame storage  OBS now using full bincount, change this
        #print(df_eq_distr)
        # Create indices for generated data
        names = ['data_nr', 'type']
        index_qubit = pd.MultiIndex.from_product([[i], np.arange(1)],
                                                 names=names)
        index_distr = pd.MultiIndex.from_product([[i], np.arange(1)+1], names=names)
    
        # Add data to Dataframes
        df_qubit = pd.DataFrame([[df_qubit.astype(np.uint8)]], index=index_qubit,
                                columns=['data'])
        df_distr = pd.DataFrame([[df_eq_distr]],
                                index=index_distr, columns=['data'])
        
        # Add dataframes to temporary list to shorten computation time

        
        ####
        pickle_eq_dist = open("defect_matrices/s9p1/dict.eq_distr","ab")
        pickle.dump(df_eq_distr,pickle_eq_dist)
        pickle_eq_dist.close()
        #####
        df_list.append(df_qubit)
        df_list.append(df_distr)
        
        # Every x iteration adds data to data file from temporary list
        # and clears temporary list

        if (i + 1) % 10000 == 0: # this needs to be sufficiently big that rsync has time to sync files before update, maybe change this to be time-based instead.
            df = df.append(df_list)
            df_list.clear()
            print('Intermediate save point reached (writing over)')
            df.to_pickle(file_path)
            print('Failed so far:', failed_syndroms)

        # If the desired amount of errors have been achieved, break the loop and finish up
        if failed_syndroms == fixed_errors:
            print('Desired amount of failes syndroms achieved, breaking loop.')
            break

    # Adds any remaining data from temporary list to data file when run is over
    if len(df_list) > 0:
        df = df.append(df_list)
        print('\nSaving all generated data (writing over)')
        df.to_pickle(file_path)

    print('\nCompleted')


if __name__ == '__main__':
    # Get job array id, working directory
    #try:
#        array_id = os.getenv('SLURM_ARRAY_TASK_ID')
#        local_dir = os.getenv('TMPDIR')
#    except:
    array_id = '10'
    local_dir = './data'
    print('Invalid environment variables, using array_id 0 and local dir.')

    params = {'code': "planar",
            'method': "STDC",
            'size': 9,
            'p_error': 0.1,#np.round((0.05 + float(array_id) / 50), decimals=2),
            'p_sampling': 0.1,#np.round((0.05 + float(array_id) / 50), decimals=2),
            'droplets':1,
            'mwpm_init':True,
            'fixed_errors':None,
            'Nc':None,
            'iters': 10,
            'conv_criteria':'error_based',
            'SEQ': 2,
            'TOPS': 10,
            'eps': 0.1}
    # Steps is a function of code size L
    params.update({'steps': int(params['size'] ** 4)})

    print('Nbr of steps to take if applicable:', params['steps'])

    # Build file path
    file_path = os.path.join(local_dir, 'data_size_'+str(params['size'])+'_method_'+params['method']+'_id_' + array_id + '_perror_' + str(params['p_error']) + '.xz')

    # Generate data
    generate(file_path, params, nbr_datapoints=5000, fixed_errors=params['fixed_errors'])

    # View data file

    '''iterator = MCMCDataReader(file_path, params['size'])
    data = iterator.full()
    for k in range(int(len(data)/2)):
        qubit_matrix = data[2*k].reshape(2,params['size'],params['size'])
        eq_distr = data[2*k+1]

        print(qubit_matrix)
        print(eq_distr)'''


Invalid environment variables, using array_id 0 and local dir.
Nbr of steps to take if applicable: 6561

DataFrame with 0 datapoints opened at: ./data/data_size_9_method_STDC_id_10_perror_0.1.xz
Starting generation of point nr: 1
Starting in MWPM state
Starting generation of point nr: 2
Starting in MWPM state
Starting generation of point nr: 3
Starting in MWPM state
Starting generation of point nr: 4
Starting in MWPM state
Starting generation of point nr: 5
Starting in MWPM state
Starting generation of point nr: 6
Starting in MWPM state
Starting generation of point nr: 7
Starting in MWPM state
Starting generation of point nr: 8
Starting in MWPM state
Starting generation of point nr: 9
Starting in MWPM state
Starting generation of point nr: 10
Starting in MWPM state
Starting generation of point nr: 11
Starting in MWPM state
Starting generation of point nr: 12
Starting in MWPM state
Starting generation of point nr: 13
Starting in MWPM state
Starting generation of point nr: 14
Starting in

Starting generation of point nr: 135
Starting in MWPM state
Starting generation of point nr: 136
Starting in MWPM state
Starting generation of point nr: 137
Starting in MWPM state
Starting generation of point nr: 138
Starting in MWPM state
Starting generation of point nr: 139
Starting in MWPM state
Starting generation of point nr: 140
Starting in MWPM state
Starting generation of point nr: 141
Starting in MWPM state
Starting generation of point nr: 142
Starting in MWPM state
Starting generation of point nr: 143
Starting in MWPM state
Starting generation of point nr: 144
Starting in MWPM state
Starting generation of point nr: 145
Starting in MWPM state
Starting generation of point nr: 146
Starting in MWPM state
Starting generation of point nr: 147
Starting in MWPM state
Starting generation of point nr: 148
Starting in MWPM state
Starting generation of point nr: 149
Starting in MWPM state
Starting generation of point nr: 150
Starting in MWPM state
Starting generation of point nr: 151
Sta

Starting generation of point nr: 273
Starting in MWPM state
Starting generation of point nr: 274
Starting in MWPM state
Starting generation of point nr: 275
Starting in MWPM state
Starting generation of point nr: 276
Starting in MWPM state
Starting generation of point nr: 277
Starting in MWPM state
Starting generation of point nr: 278
Starting in MWPM state
Starting generation of point nr: 279
Starting in MWPM state
Starting generation of point nr: 280
Starting in MWPM state
Starting generation of point nr: 281
Starting in MWPM state
Starting generation of point nr: 282
Starting in MWPM state
Starting generation of point nr: 283
Starting in MWPM state
Starting generation of point nr: 284
Starting in MWPM state
Starting generation of point nr: 285
Starting in MWPM state
Starting generation of point nr: 286
Starting in MWPM state
Starting generation of point nr: 287
Starting in MWPM state
Starting generation of point nr: 288
Starting in MWPM state
Starting generation of point nr: 289
Sta

Starting generation of point nr: 409
Starting in MWPM state
Starting generation of point nr: 410
Starting in MWPM state
Starting generation of point nr: 411
Starting in MWPM state
Starting generation of point nr: 412
Starting in MWPM state
Starting generation of point nr: 413
Starting in MWPM state
Starting generation of point nr: 414
Starting in MWPM state
Starting generation of point nr: 415
Starting in MWPM state
Starting generation of point nr: 416
Starting in MWPM state
Failed syndrom, total now: 5
Starting generation of point nr: 417
Starting in MWPM state
Starting generation of point nr: 418
Starting in MWPM state
Starting generation of point nr: 419
Starting in MWPM state
Starting generation of point nr: 420
Starting in MWPM state
Starting generation of point nr: 421
Starting in MWPM state
Starting generation of point nr: 422
Starting in MWPM state
Starting generation of point nr: 423
Starting in MWPM state
Starting generation of point nr: 424
Starting in MWPM state
Starting ge

Failed syndrom, total now: 8
Starting generation of point nr: 545
Starting in MWPM state
Starting generation of point nr: 546
Starting in MWPM state
Starting generation of point nr: 547
Starting in MWPM state
Starting generation of point nr: 548
Starting in MWPM state
Starting generation of point nr: 549
Starting in MWPM state
Starting generation of point nr: 550
Starting in MWPM state
Starting generation of point nr: 551
Starting in MWPM state
Starting generation of point nr: 552
Starting in MWPM state
Starting generation of point nr: 553
Starting in MWPM state
Starting generation of point nr: 554
Starting in MWPM state
Starting generation of point nr: 555
Starting in MWPM state
Starting generation of point nr: 556
Starting in MWPM state
Starting generation of point nr: 557
Starting in MWPM state
Starting generation of point nr: 558
Starting in MWPM state
Starting generation of point nr: 559
Starting in MWPM state
Starting generation of point nr: 560
Starting in MWPM state
Starting ge

...

Starting generation of point nr: 4439
Starting in MWPM state
Starting generation of point nr: 4440
Starting in MWPM state
Starting generation of point nr: 4441
Starting in MWPM state
Starting generation of point nr: 4442
Starting in MWPM state
Starting generation of point nr: 4443
Starting in MWPM state
Starting generation of point nr: 4444
Starting in MWPM state
Starting generation of point nr: 4445
Starting in MWPM state
Starting generation of point nr: 4446
Starting in MWPM state
Starting generation of point nr: 4447
Starting in MWPM state
Starting generation of point nr: 4448
Starting in MWPM state
Starting generation of point nr: 4449
Starting in MWPM state
Starting generation of point nr: 4450
Starting in MWPM state
Starting generation of point nr: 4451
Starting in MWPM state
Starting generation of point nr: 4452
Starting in MWPM state
Starting generation of point nr: 4453
Starting in MWPM state
Starting generation of point nr: 4454
Starting in MWPM state
Starting generation of p

Starting generation of point nr: 4573
Starting in MWPM state
Starting generation of point nr: 4574
Starting in MWPM state
Starting generation of point nr: 4575
Starting in MWPM state
Starting generation of point nr: 4576
Starting in MWPM state
Starting generation of point nr: 4577
Starting in MWPM state
Starting generation of point nr: 4578
Starting in MWPM state
Starting generation of point nr: 4579
Starting in MWPM state
Starting generation of point nr: 4580
Starting in MWPM state
Starting generation of point nr: 4581
Starting in MWPM state
Starting generation of point nr: 4582
Starting in MWPM state
Starting generation of point nr: 4583
Starting in MWPM state
Starting generation of point nr: 4584
Starting in MWPM state
Starting generation of point nr: 4585
Starting in MWPM state
Starting generation of point nr: 4586
Starting in MWPM state
Starting generation of point nr: 4587
Starting in MWPM state
Starting generation of point nr: 4588
Starting in MWPM state
Starting generation of p

Starting generation of point nr: 4707
Starting in MWPM state
Starting generation of point nr: 4708
Starting in MWPM state
Starting generation of point nr: 4709
Starting in MWPM state
Starting generation of point nr: 4710
Starting in MWPM state
Starting generation of point nr: 4711
Starting in MWPM state
Starting generation of point nr: 4712
Starting in MWPM state
Starting generation of point nr: 4713
Starting in MWPM state
Failed syndrom, total now: 85
Starting generation of point nr: 4714
Starting in MWPM state
Starting generation of point nr: 4715
Starting in MWPM state
Starting generation of point nr: 4716
Starting in MWPM state
Starting generation of point nr: 4717
Starting in MWPM state
Starting generation of point nr: 4718
Starting in MWPM state
Starting generation of point nr: 4719
Starting in MWPM state
Starting generation of point nr: 4720
Starting in MWPM state
Starting generation of point nr: 4721
Starting in MWPM state
Starting generation of point nr: 4722
Starting in MWPM 

Starting generation of point nr: 4841
Starting in MWPM state
Starting generation of point nr: 4842
Starting in MWPM state
Starting generation of point nr: 4843
Starting in MWPM state
Starting generation of point nr: 4844
Starting in MWPM state
Starting generation of point nr: 4845
Starting in MWPM state
Starting generation of point nr: 4846
Starting in MWPM state
Starting generation of point nr: 4847
Starting in MWPM state
Starting generation of point nr: 4848
Starting in MWPM state
Starting generation of point nr: 4849
Starting in MWPM state
Starting generation of point nr: 4850
Starting in MWPM state
Starting generation of point nr: 4851
Starting in MWPM state
Starting generation of point nr: 4852
Starting in MWPM state
Starting generation of point nr: 4853
Starting in MWPM state
Starting generation of point nr: 4854
Starting in MWPM state
Starting generation of point nr: 4855
Starting in MWPM state
Starting generation of point nr: 4856
Starting in MWPM state
Starting generation of p

Starting generation of point nr: 4976
Starting in MWPM state
Starting generation of point nr: 4977
Starting in MWPM state
Starting generation of point nr: 4978
Starting in MWPM state
Starting generation of point nr: 4979
Starting in MWPM state
Starting generation of point nr: 4980
Starting in MWPM state
Starting generation of point nr: 4981
Starting in MWPM state
Starting generation of point nr: 4982
Starting in MWPM state
Starting generation of point nr: 4983
Starting in MWPM state
Starting generation of point nr: 4984
Starting in MWPM state
Starting generation of point nr: 4985
Starting in MWPM state
Starting generation of point nr: 4986
Starting in MWPM state
Starting generation of point nr: 4987
Starting in MWPM state
Starting generation of point nr: 4988
Starting in MWPM state
Starting generation of point nr: 4989
Starting in MWPM state
Starting generation of point nr: 4990
Starting in MWPM state
Starting generation of point nr: 4991
Starting in MWPM state
Starting generation of p

In [11]:
iterator = MCMCDataReader(file_path, params['size'])
data = iterator.full()
for k in range(int(len(data)/2)):
    qubit_matrix = data[2*k].reshape(2,params['size'],params['size'])
    eq_distr = data[2*k+1]

    #print(qubit_matrix)
    print(eq_distr)
    print()
    #print(round(eq_distr))

[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.90000000e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.99379588e+01 6.20309298e-02 1.02751147e-05 5.98651208e-09]

[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.90000000e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.99977587e+01 2.23120686e-03 1.85986396e-07 9.94282199e-06]

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.90000000e+01
 2.66011938e-08 1.25262660e-03 1.05513314e-12 9.99987473e+01]

[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.99994213e+01 1.03042091e-05 5.68393567e-04 2.01776678e-12]

[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.99999900e+01 4.51999727e-19 9.99212695e-06 6.38112631e-11]

[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.90000000e+0

200


In [4]:
import copy  # not used
import os
import sys
import time

import pickle

import numpy as np
import pandas as pd

from src.toric_model import Toric_code
from src.planar_model import Planar_code
from src.mcmc import *
from decoders import *
from src.mwpm import *


In [74]:
file_path = 'data/data_size_5_method_STDC_id_10_perror_0.05.xz'
iterator = MCMCDataReader(file_path, 5)
data = iterator.full()
klass = []
qb = []
for k in range(int(len(data)/2)):
    qubit_matrix = data[2*k].reshape(2,5,5)
    eq_distr = data[2*k+1]

    #print(qubit_matrix)
    #print(eq_distr)
    qb.append(qubit_matrix)
    klass.append(eq_distr)

In [75]:
print(len(klass))
ek = np.array(klass)


2000


In [76]:
pickle_in1 = open("defect_matrices/s5p05MWPM/few/dict.defects_stdc","rb")
pickle_in2 = open("defect_matrices/s5p05MWPM/few/dict.eq_distr_stdc","rb")

#pickle_in1 = open("defect_matrices/s5p05MWPM/few/dict.defects_1","rb")
#pickle_in2 = open("defect_matrices/s5p05MWPM/few/dict.eq_distr_1","rb")


eq_distr = []
defects = []
while 1:
    try:
        defects.append(pickle.load(pickle_in1))
        eq_distr.append(pickle.load(pickle_in2))
    except EOFError:
        break

print(len(eq_distr))
print(len(defects))

2000
2000


In [77]:
ec = np.array(eq_distr)

In [78]:
out = np.array_equal(ec,ek)
out

True

In [69]:
if len(defects)
print(defects[1])
print()
print(qb[1])
#ek[3]

(array([[0, 0, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0]]), array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]))

[[[0 0 0 3 0]
  [0 0 0 0 0]
  [3 0 2 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [72]:
for ix in range(len(defects)):
    if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) == 0:
        continue
    #### REMOVE FOR REGULAR RUN, ONLY FOR FEW DEFECT SIM.
    if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) >3:
        continue
    
    #print(defects[ix])
    #print(qb[ix])
    #print(eq_distr[ix])

(array([[0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]))
[9.99999706e+01 1.01323424e-05 1.92732282e-05 2.87961113e-12]
(array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1]]))
[9.99988918e+01 5.66710042e-04 5.41507203e-04 3.00977713e-09]
(array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]]), array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]))
[9.99999791e+01 1.06166097e-05 1.02899447e-05 2.84729392e-12]
(array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1,